In [1]:
import numpy as np
import pandas as pd
import os
from pathlib import Path
from src.pipeline.load import filter_subjects_with_two_timepoints
from src.pipeline.connectomes import (
    merge_features_and_connectomes,
    enforce_same_subjects
)
from src.pipeline.clean import (
    handle_missing_values,
    merge_family_ids,
    enforce_common_subjects,
    drop_siblings,
    link_with_g_scores
)
from src.pipeline.preprocess import (
    preprocess_features,
    filter_sites_by_threshold,
    add_income_group,
    print_data_summary
)
from src.analysis.eda import (
    list_columns_with_missing_values,
    count_columns_with_missing_values,
    percentage_missing_values,
    divide_features,
    categorical_summary_stats
)

In [2]:
# Load data
features = pd.read_csv('../data/raw/Demographics.csv')
# Load g_score
g_factor = pd.read_csv('../data/raw/ABCD_new_G_all.csv')
# Drop "Task" since it's constant
features = features.drop(columns=["Task"], inplace=False)

# Features description
The dataset we are working with consist of 13274 rows and 20 columns. Thus, we have information for the 13274 subjects. Let us look at the description of the features:
* src_subject_id - subject identifier
* eventname - timepoint of data
* site_id_l - data collection site (may want to include this in models as a grouping factor as there are likely site effects in the data, we will sometimes model subject nested in site as random effects)
* rel_family_id - family identifier. There are some siblings, twins, etc in the data. Not necessarily an issue sometimes, but when we're doing any kind of cross validation or anything we make sure we don't have family members in different folds.
* interview_age - age of the subject in months
* Subject/Session are just recoded versions of subject ID and eventname as they match with the naming of the imaging data better
* Task - this should all be rest
* GoodRun_5 - number of good runs in the included data
* censor_5 - number of censored timepoints in the included data
* TRs - number of included timepoints in the data
* confounds_nocensor - number of confounds in the run-level nuisance correction model (other than censored timepoints). Most of these quality metrics you probably don't need to worry about
* meanFD - the mean motion of the subject during their included scans. We usually use linear and quadratic terms of this as a confound at the group level
* race.4level - subject reported race
* hisp - subject reported hispanic yes/no
* demo_sex_v2 - sex at birth 1=male, 2=female (I think I dropped any other responses, but if there are others we might need to exclude just because there are so few we can't get good estimates of effects)
* EdYearsHighest - parental years of education (highest among parents, I think I need to double check we might actually use average elsewhere, but this variable is likely not immediately relevant for now)
* IncCombinedMidpoint - combined income of parents (midpoint of a bin, because it's only reported in 10 bins)
* Income2Needs - calculated income to needs metric based on parental income and number of people in the household
* Married - parents currently married or not

In [3]:
# Keep only participants with both baseline and follow-up
df_filtered = filter_subjects_with_two_timepoints(features)

# Split by event
df_baseline = df_filtered[df_filtered["eventname"] == "baseline_year_1_arm_1"]
df_followup = df_filtered[df_filtered["eventname"] == "2_year_follow_up_y_arm_1"]

In [4]:
# Drop rows with missing values (except family IDs)
df_baseline, df_followup = handle_missing_values(df_baseline, df_followup)

# Fill in family IDs for follow-up from baseline
df_followup = merge_family_ids(df_baseline, df_followup)

In [5]:
# Enforce common subjects
df_baseline, df_followup = enforce_common_subjects(df_baseline, df_followup)

In [6]:
# Drop siblings
df_baseline = drop_siblings(df_baseline)
df_followup = drop_siblings(df_followup)


In [7]:
# Merge with g-score data
merged_df_baseline_A, merged_df_followup_A = link_with_g_scores(df_baseline, df_followup, g_factor)

In [8]:
# Define renaming rules
rename_map = {
    "G_lavaan.baseline": "g_lavaan",
    "G_lavaan.2Year": "g_lavaan",
    "demo_sex_v2": "sex",
    "interview_age": "age"
}

# Apply renaming
merged_df_baseline_A = merged_df_baseline_A.rename(columns=rename_map)
merged_df_followup_A = merged_df_followup_A.rename(columns=rename_map)

# Columns to drop
drop_cols = ["src_subject_id", "eventname", "rel_family_id",
             "Session", "GoodRun_5", "censor_5", "TRs", "confounds_nocensor"]

merged_df_baseline_A = merged_df_baseline_A.drop(columns=drop_cols)
merged_df_followup_A = merged_df_followup_A.drop(columns=drop_cols)

In [9]:
merged_df_baseline_A = add_income_group(merged_df_baseline_A, t1=50000, t2=100000)
merged_df_followup_A  = add_income_group(merged_df_followup_A, t1=50000, t2=100000)

In [18]:
A_MIN = 50

merged_df_baseline_A = filter_sites_by_threshold(
    merged_df_baseline_A, site_col="site_id_l", threshold=A_MIN
)
merged_df_followup_A = filter_sites_by_threshold(
    merged_df_followup_A, site_col="site_id_l", threshold=A_MIN
)

In [19]:
print_data_summary(merged_df_baseline_A)

Number of subjects: 2411
Age mean: 9.96 years
Age std: 0.63 years

sex:
  1.0: 1223 (50.73%)
  2.0: 1188 (49.27%)

race.4level:
  White: 1772 (73.50%)
  Other/Mixed: 360 (14.93%)
  Black: 240 (9.95%)
  Asian: 39 (1.62%)

hisp:
  No: 1979 (82.08%)
  Yes: 432 (17.92%)

income_group:
  high: 1058 (43.88%)
  medium: 723 (29.99%)
  low: 630 (26.13%)


In [20]:
print_data_summary(merged_df_followup_A)

Number of subjects: 2415
Age mean: 11.96 years
Age std: 0.65 years

sex:
  1.0: 1225 (50.72%)
  2.0: 1190 (49.28%)

race.4level:
  White: 1776 (73.54%)
  Other/Mixed: 361 (14.95%)
  Black: 239 (9.90%)
  Asian: 39 (1.61%)

hisp:
  No: 1981 (82.03%)
  Yes: 434 (17.97%)

income_group:
  high: 1211 (50.14%)
  medium: 683 (28.28%)
  low: 521 (21.57%)


In [21]:
merged_df_baseline_B = preprocess_features(features, g_factor, 'baseline_year_1_arm_1')
merged_df_followup_B = preprocess_features(features, g_factor, '2_year_follow_up_y_arm_1')

In [22]:
B_MIN = 75

merged_df_baseline_B = filter_sites_by_threshold(
    merged_df_baseline_B, site_col="site_id_l", threshold=B_MIN
)
merged_df_followup_B = filter_sites_by_threshold(
    merged_df_followup_B, site_col="site_id_l", threshold=B_MIN
)

In [23]:
print_data_summary(merged_df_baseline_B)

Number of subjects: 4321
Age mean: 9.98 years
Age std: 0.63 years

sex:
  2.0: 2217 (51.31%)
  1.0: 2104 (48.69%)

race.4level:
  White: 3071 (71.07%)
  Other/Mixed: 631 (14.60%)
  Black: 529 (12.24%)
  Asian: 90 (2.08%)

hisp:
  No: 3591 (83.11%)
  Yes: 730 (16.89%)

income_group:
  high: 1919 (44.41%)
  medium: 1244 (28.79%)
  low: 1158 (26.80%)


In [24]:
print_data_summary(merged_df_followup_B)

Number of subjects: 2432
Age mean: 11.91 years
Age std: 0.64 years

sex:
  1.0: 1235 (50.78%)
  2.0: 1197 (49.22%)

race.4level:
  White: 1731 (71.18%)
  Other/Mixed: 400 (16.45%)
  Black: 253 (10.40%)
  Asian: 48 (1.97%)

hisp:
  No: 1960 (80.59%)
  Yes: 472 (19.41%)

income_group:
  high: 1127 (46.34%)
  medium: 708 (29.11%)
  low: 597 (24.55%)


In [ ]:
# save_aligned_features(
#     merged_df_baseline_A, merged_df_followup_A,
#     merged_df_baseline_B, merged_df_followup_B
# )

Two different approaches of data preprocessing were used.
* Approach A : we consider only subjects with two time points, keep only one sibling.
* Approach B : is as close as possible to the method described in original article.

In [25]:
EVENT_TO_SESSION = {
    "baseline_year_1_arm_1": "baselineYear1Arm1",
    "2_year_follow_up_y_arm_1": "2YearFollowUpYArm1"
}

# Baseline features (Version A)
features_A_baseline_aligned, conn_A_baseline, subs_A_baseline = merge_features_and_connectomes(
    merged_df_baseline_A,
    visit=EVENT_TO_SESSION["baseline_year_1_arm_1"]
)

print(features_A_baseline_aligned.shape)
print(conn_A_baseline.shape)
print(len(subs_A_baseline))

[skip] NDARINVYXRGTMYM: Failed to convert matrix to array: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (341,) + inhomogeneous part.
Loaded 2410 subjects
Flattened shape: (2410, 87153)  (n_subjects x n_edges)
Subjects in df1 but not in df2: ['NDARINVYXRGTMYM']
Keeping 2410 common subjects.
(2410, 13)
(2410, 87153)
2410


In [26]:
# Followup features (Version A)
features_A_followup_aligned, conn_A_followup, subs_A_followup = merge_features_and_connectomes(
    merged_df_followup_A,
    visit=EVENT_TO_SESSION["2_year_follow_up_y_arm_1"]
)

print(features_A_followup_aligned.shape)
print(conn_A_followup.shape)
print(len(subs_A_followup))

Loaded 2415 subjects
Flattened shape: (2415, 87153)  (n_subjects x n_edges)
Keeping 2415 common subjects.
(2415, 13)
(2415, 87153)
2415


In [29]:
(features_A_baseline_final, conn_A_baseline_final,
 features_A_followup_final, conn_A_followup_final,
 subs_A_final) = enforce_same_subjects(
    features_A_baseline_aligned, conn_A_baseline, subs_A_baseline,
    features_A_followup_aligned, conn_A_followup, subs_A_followup
)

In [32]:
output_dir = Path("data/processed/version_A")
output_dir.mkdir(parents=True, exist_ok=True)

# Save features
features_A_baseline_final.to_csv(output_dir / "features_baseline.csv", index=False)
features_A_followup_final.to_csv(output_dir / "features_followup.csv", index=False)

# Save connectomes
np.save(output_dir / "connectomes_baseline.npy", conn_A_baseline_final)
np.save(output_dir / "connectomes_followup.npy", conn_A_followup_final)

# Save subject IDs
pd.DataFrame({"Subject": subs_A_final}).to_csv(output_dir / "subjects.csv", index=False)


In [33]:
for fname in ["features_baseline.csv", "features_followup.csv",
              "connectomes_baseline.npy", "connectomes_followup.npy",
              "subjects.csv"]:
    fpath = output_dir / fname
    print(f"{fname}: exists={fpath.exists()}, size={os.path.getsize(fpath) if fpath.exists() else 'N/A'} bytes")


features_baseline.csv: exists=True, size=323164 bytes
features_followup.csv: exists=True, size=323305 bytes
connectomes_baseline.npy: exists=True, size=1679612744 bytes
connectomes_followup.npy: exists=True, size=1679612744 bytes
subjects.csv: exists=True, size=38552 bytes


In [34]:
# Baseline features (Version B)
features_B_baseline_aligned, conn_B_baseline, subs_B_baseline = merge_features_and_connectomes(
    merged_df_baseline_B,
    visit=EVENT_TO_SESSION["baseline_year_1_arm_1"]
)

print(features_B_baseline_aligned.shape)
print(conn_B_baseline.shape)
print(len(subs_B_baseline))

Loaded 4321 subjects
Flattened shape: (4321, 87153)  (n_subjects x n_edges)
Keeping 4321 common subjects.
(4321, 13)
(4321, 87153)
4321


In [36]:
# Followup features (Version B)
features_B_followup_aligned, conn_B_followup, subs_B_followup = merge_features_and_connectomes(
    merged_df_followup_B,
    visit=EVENT_TO_SESSION["2_year_follow_up_y_arm_1"]
)

print(features_B_followup_aligned.shape)
print(conn_B_followup.shape)
print(len(subs_B_followup))

Loaded 2432 subjects
Flattened shape: (2432, 87153)  (n_subjects x n_edges)
Keeping 2432 common subjects.
(2432, 13)
(2432, 87153)
2432


In [37]:
output_dir = Path("data/processed/version_B")
output_dir.mkdir(parents=True, exist_ok=True)

# Save features
features_B_baseline_aligned.to_csv(output_dir / "features_baseline.csv", index=False)
features_B_followup_aligned.to_csv(output_dir / "features_followup.csv", index=False)

# Save connectomes
np.save(output_dir / "connectomes_baseline.npy", conn_B_baseline)
np.save(output_dir / "connectomes_followup.npy", conn_B_followup)

# Save baseline subjects
pd.DataFrame({"Subject": subs_B_baseline}).to_csv(output_dir / "subjects_baseline.csv", index=False)

# Save followup subjects
pd.DataFrame({"Subject": subs_B_followup}).to_csv(output_dir / "subjects_followup.csv", index=False)